## Interaktiv Phi 3 Mini 4K Instruksjonschatbot med Whisper

### Introduksjon:
Den interaktive Phi 3 Mini 4K instruksjonschatboten er et verktøy som lar brukere samhandle med Microsoft Phi 3 Mini 4K instruksjonsdemo ved hjelp av tekst- eller lydinnspill. Chatboten kan brukes til en rekke oppgaver, som oversettelse, værmeldinger og generell informasjonsinnhenting.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Opprett din Huggingface Access Token](https://huggingface.co/settings/tokens)

Opprett en ny token  
Gi den et nytt navn  
Velg skrive-tillatelser  
Kopier token og lagre den på et trygt sted


Den følgende Python-koden utfører to hovedoppgaver: importering av `os`-modulen og setting av en miljøvariabel.

1. Importering av `os`-modulen:
   - `os`-modulen i Python gir en måte å interagere med operativsystemet på. Den lar deg utføre ulike oppgaver relatert til operativsystemet, som å få tilgang til miljøvariabler, jobbe med filer og kataloger, osv.
   - I denne koden blir `os`-modulen importert ved hjelp av `import`-setningen. Denne setningen gjør funksjonaliteten til `os`-modulen tilgjengelig for bruk i det gjeldende Python-skriptet.

2. Setting av en miljøvariabel:
   - En miljøvariabel er en verdi som kan nås av programmer som kjører på operativsystemet. Det er en måte å lagre konfigurasjonsinnstillinger eller annen informasjon som kan brukes av flere programmer.
   - I denne koden blir en ny miljøvariabel satt ved hjelp av `os.environ`-ordboken. Nøkkelen i ordboken er `'HF_TOKEN'`, og verdien blir tildelt fra variabelen `HUGGINGFACE_TOKEN`.
   - Variabelen `HUGGINGFACE_TOKEN` er definert rett over denne kodebiten, og den blir tildelt en strengverdi `"hf_**************"` ved hjelp av `#@param`-syntaksen. Denne syntaksen brukes ofte i Jupyter-notebooks for å tillate brukerinput og parameterkonfigurasjon direkte i notebook-grensesnittet.
   - Ved å sette miljøvariabelen `'HF_TOKEN'` kan den nås av andre deler av programmet eller andre programmer som kjører på det samme operativsystemet.

Oppsummert importerer denne koden `os`-modulen og setter en miljøvariabel med navnet `'HF_TOKEN'` med verdien som er angitt i variabelen `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Denne kodebiten definerer en funksjon kalt clear_output som brukes til å fjerne utdataene fra den nåværende cellen i Jupyter Notebook eller IPython. La oss bryte ned koden og forstå dens funksjonalitet:

Funksjonen clear_output tar én parameter kalt wait, som er en boolsk verdi. Som standard er wait satt til False. Denne parameteren avgjør om funksjonen skal vente til nye utdata er tilgjengelige for å erstatte de eksisterende utdataene før de fjernes.

Selve funksjonen brukes til å fjerne utdataene fra den nåværende cellen. I Jupyter Notebook eller IPython, når en celle produserer utdata, som for eksempel tekst som er skrevet ut eller grafiske diagrammer, vises disse utdataene under cellen. Funksjonen clear_output lar deg fjerne disse utdataene.

Implementeringen av funksjonen er ikke gitt i kodebiten, som indikert av ellipsen (...). Ellipsen representerer en plassholder for den faktiske koden som utfører fjerningen av utdataene. Implementeringen av funksjonen kan innebære interaksjon med Jupyter Notebook- eller IPython-APIet for å fjerne de eksisterende utdataene fra cellen.

Alt i alt gir denne funksjonen en praktisk måte å fjerne utdataene fra den nåværende cellen i Jupyter Notebook eller IPython, noe som gjør det enklere å administrere og oppdatere de viste utdataene under interaktive kodingsøkter.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Utfør tekst-til-tale (TTS) ved hjelp av Edge TTS-tjenesten. La oss gå gjennom de relevante funksjonsimplementasjonene én etter én:

1. `calculate_rate_string(input_value)`: Denne funksjonen tar en inputverdi og beregner hastighetsstrengen for TTS-stemmen. Inputverdien representerer ønsket talehastighet, der en verdi på 1 representerer normal hastighet. Funksjonen beregner hastighetsstrengen ved å trekke 1 fra inputverdien, multiplisere den med 100, og deretter bestemme fortegnet basert på om inputverdien er større enn eller lik 1. Funksjonen returnerer hastighetsstrengen i formatet "{sign}{rate}".

2. `make_chunks(input_text, language)`: Denne funksjonen tar en inputtekst og et språk som parametere. Den deler opp inputteksten i deler basert på språkspesifikke regler. I denne implementasjonen, hvis språket er "English", deler funksjonen teksten ved hvert punktum (".") og fjerner eventuelle innledende eller avsluttende mellomrom. Den legger deretter til et punktum på slutten av hver del og returnerer den filtrerte listen med deler.

3. `tts_file_name(text)`: Denne funksjonen genererer et filnavn for TTS-lydfilen basert på inputteksten. Den utfører flere transformasjoner på teksten: fjerner et avsluttende punktum (hvis til stede), konverterer teksten til små bokstaver, fjerner innledende og avsluttende mellomrom, og erstatter mellomrom med understreker. Deretter trunkerer den teksten til maksimalt 25 tegn (hvis lengre) eller bruker hele teksten hvis den er tom. Til slutt genererer den en tilfeldig streng ved hjelp av [`uuid`]-modulen og kombinerer den med den trunkerte teksten for å lage filnavnet i formatet "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Denne funksjonen slår sammen flere lydfiler til én enkelt lydfil. Den tar en liste med lydfilbaner og en utbanesti som parametere. Funksjonen initialiserer et tomt `AudioSegment`-objekt kalt [`merged_audio`]. Deretter itererer den gjennom hver lydfilbane, laster lydfilen ved hjelp av `AudioSegment.from_file()`-metoden fra `pydub`-biblioteket, og legger til den aktuelle lydfilen i [`merged_audio`]-objektet. Til slutt eksporterer den den sammenslåtte lyden til den spesifiserte utbanen i MP3-format.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Denne funksjonen utfører TTS-operasjonen ved hjelp av Edge TTS-tjenesten. Den tar en liste med tekstdeler, talehastigheten, stemmens navn og lagringsbanen som parametere. Hvis antallet deler er større enn 1, oppretter funksjonen en katalog for lagring av de individuelle lydfilene for hver del. Deretter itererer den gjennom hver del, konstruerer en Edge TTS-kommando ved hjelp av `calculate_rate_string()`-funksjonen, stemmens navn og tekstdelen, og utfører kommandoen ved hjelp av `os.system()`-funksjonen. Hvis kommandoutførelsen er vellykket, legger den til banen til den genererte lydfilen i en liste. Etter å ha behandlet alle delene, slår den sammen de individuelle lydfilene ved hjelp av `merge_audio_files()`-funksjonen og lagrer den sammenslåtte lyden til den spesifiserte lagringsbanen. Hvis det bare er én del, genererer den direkte Edge TTS-kommandoen og lagrer lyden til lagringsbanen. Til slutt returnerer den lagringsbanen til den genererte lydfilen.

6. `random_audio_name_generate()`: Denne funksjonen genererer et tilfeldig lydfilnavn ved hjelp av [`uuid`]-modulen. Den genererer en tilfeldig UUID, konverterer den til en streng, tar de første 8 tegnene, legger til ".mp3"-utvidelsen, og returnerer det tilfeldige lydfilnavnet.

7. `talk(input_text)`: Denne funksjonen er hovedinngangspunktet for å utføre TTS-operasjonen. Den tar en inputtekst som parameter. Den sjekker først lengden på inputteksten for å avgjøre om det er en lang setning (større enn eller lik 600 tegn). Basert på lengden og verdien av variabelen `translate_text_flag`, bestemmer den språket og genererer listen med tekstdeler ved hjelp av `make_chunks()`-funksjonen. Deretter genererer den en lagringsbane for lydfilen ved hjelp av `random_audio_name_generate()`-funksjonen. Til slutt kaller den `edge_free_tts()`-funksjonen for å utføre TTS-operasjonen og returnerer lagringsbanen til den genererte lydfilen.

Samlet sett arbeider disse funksjonene sammen for å dele opp inputteksten i deler, generere et filnavn for lydfilen, utføre TTS-operasjonen ved hjelp av Edge TTS-tjenesten, og slå sammen de individuelle lydfilene til én enkelt lydfil.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementeringen av to funksjoner: convert_to_text og run_text_prompt, samt deklarasjonen av to klasser: str og Audio.

Funksjonen convert_to_text tar en audio_path som input og transkriberer lyd til tekst ved hjelp av en modell kalt whisper_model. Funksjonen sjekker først om gpu-flagget er satt til True. Hvis det er tilfelle, brukes whisper_model med visse parametere som word_timestamps=True, fp16=True, language='English', og task='translate'. Hvis gpu-flagget er False, brukes whisper_model med fp16=False. Den resulterende transkripsjonen lagres deretter i en fil kalt 'scan.txt' og returneres som tekst.

Funksjonen run_text_prompt tar en melding og en chat_history som input. Den bruker phi_demo-funksjonen til å generere et svar fra en chatbot basert på inputmeldingen. Det genererte svaret sendes deretter til talk-funksjonen, som konverterer svaret til en lydfil og returnerer filbanen. Audio-klassen brukes til å vise og spille av lydfilen. Lyden vises ved hjelp av display-funksjonen fra IPython.display-modulen, og Audio-objektet opprettes med autoplay=True-parameteren, slik at lyden starter automatisk. Chat_history oppdateres med inputmeldingen og det genererte svaret, og en tom streng samt den oppdaterte chat_history returneres.

Str-klassen er en innebygd klasse i Python som representerer en sekvens av tegn. Den tilbyr ulike metoder for å manipulere og arbeide med strenger, som capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, og flere. Disse metodene lar deg utføre operasjoner som søk, erstatning, formatering og manipulering av strenger.

Audio-klassen er en tilpasset klasse som representerer et lydobjekt. Den brukes til å lage en lydspiller i Jupyter Notebook-miljøet. Klassen aksepterer ulike parametere som data, filename, url, embed, rate, autoplay og normalize. Data-parameteren kan være en numpy-array, en liste med prøver, en streng som representerer et filnavn eller URL, eller rå PCM-data. Filename-parameteren brukes til å spesifisere en lokal fil for å laste inn lyddata fra, og url-parameteren brukes til å spesifisere en URL for å laste ned lyddata fra. Embed-parameteren avgjør om lyddataene skal integreres ved hjelp av en data-URI eller refereres fra den opprinnelige kilden. Rate-parameteren spesifiserer samplingsfrekvensen for lyddataene. Autoplay-parameteren avgjør om lyden skal starte automatisk. Normalize-parameteren spesifiserer om lyddataene skal normaliseres (skaleres) til det maksimale mulige området. Audio-klassen tilbyr også metoder som reload for å laste inn lyddata på nytt fra fil eller URL, og attributter som src_attr, autoplay_attr og element_id_attr for å hente de tilsvarende attributtene for lydelementet i HTML.

Samlet sett brukes disse funksjonene og klassene til å transkribere lyd til tekst, generere lydsvar fra en chatbot, og vise og spille av lyd i Jupyter Notebook-miljøet.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
